# Measuring the speed and accuracy for all combinations of page detectors, screen detectors, and scene detectors

In [1]:
from argparse import Namespace
from contextlib import redirect_stderr
from datetime import datetime, timedelta
from itertools import product
from pathlib import Path
from string import Template
import os

import pandas as pd
from IPython.display import display
with open(os.devnull, 'w') as f:
    with redirect_stderr(f):
        import keras
from tqdm import tqdm
from video699.__main__ import PAGE_DETECTOR_NAMES, SCREEN_DETECTOR_NAMES, SCENE_DETECTOR_NAMES, _screen_event_detector

from annotated import VIDEO_FILENAME, DOCUMENT_FILENAME, evaluate_event_detector

COLUMN_NAMES = ['Page detector', 'Screen detector', 'Scene detector']

In [2]:
# %env CUDA_VISIBLE_DEVICES=
# ^ uncomment to disable GPU in TensorFlow and PyTorch
def duration_and_accuracy(parameters):
    page_detector_name, screen_detector_name, scene_detector_name = parameters
    start_time = datetime.now()
    directory = Path('speed_and_accuracy_outputs')
    directory.mkdir(exist_ok=True)
    output = directory / Path(VIDEO_FILENAME).with_suffix('').name
    output = Path(f'{output}-{page_detector_name}-{screen_detector_name}-{scene_detector_name}')
    accuracy_output = output.with_suffix('.accuracy')
    duration_output = output.with_suffix('.duration')
    accuracy_exists = False
    duration_exists = False
    try:
        with accuracy_output.open('rt') as af:
            accuracy_exists = True
            accuracy = float(af.read())
    except IOError:
        pass
    try:
        with duration_output.open('rt') as df:
            duration_exists = True
            duration = timedelta(seconds=float(df.read()))
    except IOError:
        pass
    if not accuracy_exists or not duration_exists:
        args = Namespace(**{
            'convex_quadrangle_tracker': 'rtree_deque',
            'screen_detector': screen_detector_name,
            'scene_detector': scene_detector_name,
            'page_detector': page_detector_name,
            'institution': 'fimu',
            'room': 'd2',
            'camera': 'default_2004',
            'date': '2019-11-12T00:00:00+01:00',
            'documents': [DOCUMENT_FILENAME],
            'video': VIDEO_FILENAME,
            'quiet': True,
        })
        screen_event_detector = _screen_event_detector(args)
        num_successes, num_trials = evaluate_event_detector(screen_event_detector)
        finish_time = datetime.now()
        duration = finish_time - start_time
        accuracy = 100.0 * num_successes / num_trials
        if not accuracy_exists:
            with accuracy_output.open('wt') as af:
                print(accuracy, file=af)
        if not duration_exists:
            with duration_output.open('wt') as df:
                print(duration.total_seconds(), file=df)
    return (duration, accuracy)

In [3]:
parameters_list = list(product(PAGE_DETECTOR_NAMES, SCREEN_DETECTOR_NAMES, SCENE_DETECTOR_NAMES))
results = map(duration_and_accuracy, tqdm(parameters_list))
durations, accuracies = zip(*results)

100%|██████████| 16/16 [3:15:19<00:00, 732.50s/it]   


In [4]:
index = pd.MultiIndex.from_tuples(parameters_list, names=COLUMN_NAMES)
df = pd.Series(map(lambda d: 4546.0 / d.total_seconds(), durations), index=index).unstack(level=0)
df = df[PAGE_DETECTOR_NAMES]
df = df.T[SCREEN_DETECTOR_NAMES].T
df.T.style.set_caption('Speeds').format('{:,.2f}×').background_gradient(cmap='RdYlGn', axis=None)

In [5]:
index = pd.MultiIndex.from_tuples(parameters_list, names=COLUMN_NAMES)
df = pd.Series(accuracies, index=index).unstack(level=0)
df = df[PAGE_DETECTOR_NAMES]
df = df.T[SCREEN_DETECTOR_NAMES].T
df.T.style.set_caption('Accuracies').format('{:,.2f}%').background_gradient(cmap='RdYlGn', axis=None)